In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import h5py

In [23]:
batch_size = 32  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 531  # Number of samples to train on.

In [2]:
pwd

'/Users/tatiana/Documents/Uni/Thesis/yakut_thesis/DL'

In [5]:
data_path2 = "../FST/eval_full/sah_full.dev"

cols =["LEMMA", "WORD", "POS"]
import pandas as pd
df = pd.read_csv(data_path2, sep="\t", names = cols, encoding="utf-8")

,LEMMA,WORD,POS
0,бу,бу,<DET>
1,аайы,аайы,<DET>
2,бу,маны,<DET>|<Case=Acc>
3,яблоко,яблокаттан,<NOUN>|<Number=Sing>|<Case=Abl>
4,ыстаал,ыстаалтан,<NOUN>|<Number=Sing>|<Case=Abl>
...,...,...,...
664,ыҥыр,ыҥырар,<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>
665,гын,гынар,<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>
666,уст,устар,<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>
667,күл,күлэ,<VERB>|<Tense=Pres>|<VerbForm=Trans>


In [4]:
df.LEMMA = df.LEMMA + df.POS
df = df[["LEMMA", "WORD", "POS"]]

In [5]:
df = df.drop("POS", axis=1)
df

,LEMMA,WORD
0,бу<DET>,бу
1,аайы<DET>,аайы
2,бу<DET>|<Case=Acc>,маны
3,яблоко<NOUN>|<Number=Sing>|<Case=Abl>,яблокаттан
4,ыстаал<NOUN>|<Number=Sing>|<Case=Abl>,ыстаалтан
...,...,...
664,ыҥыр<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>,ыҥырар
665,гын<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>,гынар
666,уст<VERB>|<Number=Sing>|<Person=3>|<Tense=Pres>,устар
667,күл<VERB>|<Tense=Pres>|<VerbForm=Trans>,күлэ


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(df["WORD"].to_list(), df["LEMMA"].to_list(), train_size=0.8, shuffle=True)

In [7]:
train = list(zip(X_train,y_train))
train = pd.DataFrame(train, columns=['FORM','POS'])
train.to_csv("train.dev", encoding="utf-8", sep="\t", index=False, header=False)

In [8]:
valid = list(zip(X_valid,y_valid))
valid = pd.DataFrame(valid, columns=['FORM','POS'])
valid.to_csv("valid.test", encoding="utf-8", sep="\t", index=False, header=False)

In [9]:
train

,FORM,POS
0,биир,биир<NUM>|<NumType=Card>
1,илиигэ,илии<NOUN>|<Number=Sing>|<Case=Dat>
2,суон,суон<ADJ>
3,лааппы,лааппы<NOUN>|<Number=Sing>|<Case=Nom>
4,хотой,хотой<NOUN>|<Number=Sing>|<Case=Nom>
...,...,...
530,тиэйиэхтэрэ,тиэй<VERB>|<Number=Plur>|<Person=3>|<Tense=Fut>
531,үөрэнэҕит,үөрэн<VERB>|<Number=Plur>|<Person=2>|<Tense=Pres>
532,кыыс,кыыс<NOUN>|<Number=Sing>|<Case=Nom>
533,кырдьаҕас,кырдьаҕас<ADJ>


In [10]:
valid

,FORM,POS
0,сууннум,сууй<VERB>|<Number=Sing>|<Person=1>|<Tense=Nea...
1,баарый,баар<AUX>|<Int=Yes>
2,барда,бар<VERB>|<Number=Sing>|<Person=3>|<Tense=Near...
3,ханна,ханна<PRON>|<PronType=Int>
4,ыл,ыл<VERB>|<Mood=Imp>|<Number=Sing>|<Person=2>|<...
...,...,...
129,айаана,айаана<PROPN>
130,дьоннор,дьон<NOUN>|<Number=Plur>|<Case=Nom>
131,харандааһы,харандаас<NOUN>|<Number=Sing>|<Case=Acc>
132,аһыыллар,аһаа<VERB>|<Number=Plur>|<Person=3>|<Tense=Pres>


In [12]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open("train.dev", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text  = line.split('\t')
    #print(input_text)
    
    #x_train, x_valid, y_train, y_valid = train_test_split(input_texts, target_texts, test_size=0.33, shuffle= True)

    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    #target_text = target_text# + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 531
Number of unique input tokens: 32
Number of unique output tokens: 83
Max sequence length for inputs: 14
Max sequence length for outputs: 78


In [13]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


In [14]:
#test 

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[char]] = 1.
    decoder_target_data[i, t:, target_token_index[char]] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [15]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
14/14 [==============================] - 3s 121ms/step - loss: 2.8464 - accuracy: 0.4660 - val_loss: 2.6467 - val_accuracy: 0.4653
Epoch 2/100
14/14 [==============================] - 1s 101ms/step - loss: 2.2809 - accuracy: 0.5012 - val_loss: 2.2222 - val_accuracy: 0.4872
Epoch 3/100
14/14 [==============================] - 1s 98ms/step - loss: 2.1780 - accuracy: 0.5157 - val_loss: 2.2593 - val_accuracy: 0.5064
Epoch 4/100
14/14 [==============================] - 1s 97ms/step - loss: 2.0191 - accuracy: 0.5291 - val_loss: 2.1385 - val_accuracy: 0.5237
Epoch 5/100
14/14 [==============================] - 1s 101ms/step - loss: 1.8465 - accuracy: 0.5413 - val_loss: 1.7541 - val_accuracy: 0.5147
Epoch 6/100
14/14 [==============================] - 1s 100ms/step - loss: 1.6658 - accuracy: 0.5645 - val_loss: 1.6385 - val_accuracy: 0.5572
Epoch 7/100
14/14 [==============================] - 1s 97ms/step - loss: 1.5496 - accuracy: 0.6073 - val_loss: 1.5865 - val_accuracy: 0.6051
Ep

Epoch 58/100
14/14 [==============================] - 2s 119ms/step - loss: 0.0780 - accuracy: 0.9764 - val_loss: 0.2095 - val_accuracy: 0.9473
Epoch 59/100
14/14 [==============================] - 2s 119ms/step - loss: 0.0710 - accuracy: 0.9785 - val_loss: 0.2195 - val_accuracy: 0.9438
Epoch 60/100
14/14 [==============================] - 2s 114ms/step - loss: 0.0697 - accuracy: 0.9791 - val_loss: 0.2122 - val_accuracy: 0.9426
Epoch 61/100
14/14 [==============================] - 2s 116ms/step - loss: 0.0706 - accuracy: 0.9796 - val_loss: 0.2125 - val_accuracy: 0.9456
Epoch 62/100
14/14 [==============================] - 2s 116ms/step - loss: 0.0634 - accuracy: 0.9814 - val_loss: 0.2148 - val_accuracy: 0.9433
Epoch 63/100
14/14 [==============================] - 2s 127ms/step - loss: 0.0625 - accuracy: 0.9814 - val_loss: 0.2209 - val_accuracy: 0.9412
Epoch 64/100
14/14 [==============================] - 2s 115ms/step - loss: 0.0582 - accuracy: 0.9831 - val_loss: 0.2344 - val_accuracy:

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 32)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 83)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        295936      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [17]:
# Generalization metrics
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data, verbose=0)
print(f'Score is: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')


Score is: loss of 0.06938628852367401; accuracy of 98.45960736274719%


In [18]:
# Save model
model.save('sah.h5')

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [19]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [20]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: биир
Decoded sentence: биир<NUM>|<NumType=Card>

-
Input sentence: илиигэ
Decoded sentence: илии<NOUN>|<Number=Sing>|<Case=Nom>

-
Input sentence: суон
Decoded sentence: суон<ADJ>

-
Input sentence: лааппы
Decoded sentence: лааппы<NOUN>|<Number=Sing>|<Case=Nom>

-
Input sentence: хотой
Decoded sentence: хотой<NOUN>|<Number=Sing>|<Case=Nom>

-
Input sentence: оҕонньор
Decoded sentence: оҕонньор<NOUN>|<Number=Sing>|<Case=Nom>

-
Input sentence: сарсын
Decoded sentence: сарсын<ADV>

-
Input sentence: сыаны
Decoded sentence: сыа<NOUN>|<Number=Sing>|<Case=Acc>

-
Input sentence: кэппэт
Decoded sentence: кэт<VERB>|<Number=Sing>|<Person=2>|<Tense=PastPerf>

-
Input sentence: эдьиийим
Decoded sentence: эдьиий<NOUN>|<Number=Sing>|<Number[psor]=Sing>|<Person[psor]=3>|<Case=Nom>



In [21]:
val_input_texts = [] # valid["FORM"].to_list()
val_target_texts = [] # valid["POS"].to_list()
line_ix = len(valid)

with open("valid.test", 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text  = line.split('\t')
        val_input_texts.append(input_text)
        val_target_texts.append(target_text)

val_encoder_input_data = np.zeros((len(val_input_texts), max([len(txt) for txt in val_input_texts]),
   num_encoder_tokens), dtype='float32')

try:
    for i, input_text in enumerate(val_input_texts):
        for t, char in enumerate(input_text):
        #print(val_encoder_input_data[i,t])
            val_encoder_input_data[i, t, input_token_index[char]] = 1.

except KeyError:
    pass

In [22]:
for seq_index in range(10):
    input_seq = val_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input word:', val_input_texts[seq_index])
    print('Decoded feature:', decoded_sentence[:-1])
    print('Correct feature:', val_target_texts[seq_index])

-
Input word: сууннум
Decoded feature: нуун<ADJ>
Correct feature: сууй<VERB>|<Number=Sing>|<Person=1>|<Tense=NearPast>
-
Input word: баарый
Decoded feature: бар<VERB>|<Number=Sing>|<Person=2>|<Tense=PastPerf>
Correct feature: баар<AUX>|<Int=Yes>
-
Input word: барда
Decoded feature: бар<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>
Correct feature: бар<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>
-
Input word: ханна
Decoded feature: ханнас<NOUN>|<Number=Sing>|<Case=Dat>
Correct feature: ханна<PRON>|<PronType=Int>
-
Input word: ыл
Decoded feature: ыл<VERB>|<Number=Sing>|<Person=3>|<Tense=NearPast>
Correct feature: ыл<VERB>|<Mood=Imp>|<Number=Sing>|<Person=2>|<Tense=Pres>
-
Input word: ыйыстыбыт
Decoded feature: ыйаа<VERB>|<Number=Plur>|<Person=3>|<Tense=Pres>
Correct feature: ыйыһын<VERB>|<Number=Sing>|<Person=3>|<Tense=PastResultI>
-
Input word: кэлбэккит
Decoded feature: кэл<VERB>|<Number=Sing>|<Person=3>|<Tense=PastPerf>
Correct feature: кэл<VERB>|<Number=Plur>|<Person=2>|<Polari